In [1]:
import numpy as np
import random

In [2]:
def create_tiles():
    # create tiles

    # regular tiles
    tiles = []
    for i in range(1,10):
        for j in ['C', 'D', 'B']:
            tiles.append((j, i))   
    tiles = tiles*4

    # winds
    for i in range(1,5):
        for j in ['WN', 'WS', 'WE', 'WW', 'WG']:
            tiles.append((j, i)) 

    # sort
    tiles.sort()

    # @TODO - add more?
    
    return tiles

In [3]:
class Wall:
    def __init__(self, tiles):
        self.tiles = tiles

In [4]:
class Player:
    def __init__(self, Name):
        self.name = Name
        self.hand = []
        self.revealed = []
        self.trash = []
        
    def throw(self):      
        # print players hand
        print('Hand:')
        for s in ['B', 'C', 'D']:
            print([tile for tile in self.hand if tile[0].find(s) != -1])
        print('\n')
            
        print('Revealed:')
        print(self.revealed)
        print('\n')

        # throw out tile
        r = input("Which tile to throw out?")

        # delete from hand
        index = [i for i, tile in enumerate(self.hand) if tile[0]==r[0] and tile[1]==int(r[1])][0]
        throw_tile = self.hand[index]
        del self.hand[index]
        
        return throw_tile
    

    def draw(self, draw_tile):
        self.hand.append(draw_tile)
        self.hand.sort()
        print(''' 
             _____
            |  {}  |
            |     |
            |  {}  |
            |_____|'''.format(draw_tile[0], draw_tile[1]))

        

class Wall:
    def __init__(self, tiles):
        self.tiles = tiles
        

class MahJong:
    def __init__(self, player_names, tiles):
        
        print('MAHJONG - created by Tom Sharp')
        print('\n')
        assert len(player_names) == 4, 'Must have exactly four players'
        
        print('SETTING UP MAHJONG TABLE')
        # shuffle seating
        random.shuffle(player_names)
        self.players = [Player(name) for name in player_names]
        print('Seating Order:', [p.name for p in self.players])
        
        self.tiles = tiles
        print('Tiles:', tiles, '\n')
        
        self.game = None
        
        
    def create_game(self):
        self.game = self.Game()
        
        

    class Game:
        def __init__(self):
            print('CREATING GAME')
            print('Randomly shuffling tiles, building wall...')
            print('\n')
            [random.shuffle(mahjong.tiles) for i in range(1000)]
            self.wall = Wall(mahjong.tiles)
            self.players = mahjong.players
            self.pool = []

        def deal_tiles(self):
            print('DEALING TILES')
            print('Dealing tiles to each player, removing winds...')
            print('\n')
            # deal out tiles
            for k in range(0, 4):
                for i in range(0, 4):
                    j = (k*16) + (i*4)
                    tiles = self.wall.tiles[j:(j+4)]
                    self.players[i].hand.extend(tiles)

            # dealer gets extra piece
            self.players[0].hand.append(self.wall.tiles[64])

            # update wall 
            self.wall.tiles = self.wall.tiles[65:]


            # REPLACE WIND TILES
            while any(s.find('W')==0 for s in [tile[0] \
                      for player in self.players for tile in player.hand]):
                for i in range(0,4):      

                    # throw out wind tiles
                    wind_indexes = [i for i, tile in enumerate(self.players[i].hand) if tile[0].find('W') == 0]
                    for index in sorted(wind_indexes, reverse=True):
                        self.players[i].trash.append(self.players[i].hand[index])
                        del self.players[i].hand[index]


                    if len(wind_indexes) >  0:
                        # draw new tiles
                        new_tiles = self.wall.tiles[-len(wind_indexes):]
                        self.players[i].hand.extend(new_tiles)

                        # update wall
                        self.wall.tiles = self.wall.tiles[:-len(wind_indexes)]
                    
            # sort every hand
            [player.hand.sort() for player in self.players]
            
        
        def reveal_joker(self):
            print('JOKER')
            self.joker = self.wall.tiles[-1]
            
            # update wall
            self.wall.tiles = self.wall.tiles[:-1]
            
            print(''' 
                 _____
                |  {}  |
                |     |
                |  {}  |
                |_____|'''.format(self.joker[0], self.joker[1]))
            
                    
        
        def start(self):
            print('STARTING GAME')
            print('Dealer ({}): Throw out first tile...'.format(self.players[0].name))
            self.current_player = self.players[0]
            
            # throw event
            print('Joker:')
            print(self.joker)
            print('\n')
            thrown_tile = self.current_player.throw()
            self.pool.append(thrown_tile)
            print('{} throws out:'.format(self.current_player.name))
            print(''' 
             _____
            |  {}  |
            |     |
            |  {}  |
            |_____|'''.format(thrown_tile[0], thrown_tile[1]))
            print('\n')

            
            advance_turn = True
            while advance_turn == True:
                    
                # check pong
                pong = False
                for player in self.players:
                    matches = [i for i, tile in enumerate(player.hand) if tile[0]==thrown_tile[0] and tile[1]==int(thrown_tile[1])]
                    if len(matches)==2:
                        pong = True
                        print("==============================================")
                        print('PONG!', player.name)
                        # add to player's revealed, remove from hand
                        player.revealed.append(thrown_tile*3)
                        for index in sorted(matches, reverse=True):
                            del player.hand[index]
                        
                        # update current player
                        self.current_player = player
                
                if pong == False:
                    i = [i for i, player in enumerate(mahjong.game.players) if player == mahjong.game.current_player][0]
                    if i == 3:
                        i = 0
                    else:
                        i = i+1
                    self.current_player = self.players[i]
                    
                    print("==============================================")
                    print("{}'s TURN".format(self.current_player.name))
                    # check if can eat
                    
                    can_eat = False
                    nums = [tile[1] for tile in mahjong.game.current_player.hand if tile[0]==mahjong.game.pool[-1][0]]
                    for pair in [(mahjong.game.pool[-1][1] - 1 , mahjong.game.pool[-1][1] + 1),
                                 (mahjong.game.pool[-1][1] - 2 , mahjong.game.pool[-1][1] - 1),
                                 (mahjong.game.pool[-1][1] + 2 , mahjong.game.pool[-1][1] + 1)]:
                        if pair[0] in nums and pair[1] in nums:
                            can_eat = True
                            
                    if can_eat==True:
                        print('Pool:')
                        print(self.pool)
                        print('\n')
                        
                        print('Hand:')
                        for s in ['B', 'C', 'D']:
                            print([tile for tile in self.current_player.hand if tile[0].find(s) != -1])
                        print('\n')

                        print('Revealed:')
                        print(self.current_player.revealed)
                        print('\n')
                        
                        eat = input("Eat? Y/N")
                        if eat == 'Y':
                            first_tile = input('First tile?')
                            second_tile = input('Second tile?')
                            tiles = [first_tile, second_tile]
                            print(tiles[0], tiles[1])
                            self.current_player.revealed.append((tiles[0][0], int(tiles[0][1])))
                            self.current_player.revealed.append((tiles[1][0], int(tiles[1][1])))
                            self.current_player.revealed.append(self.pool[-1])
                            self.pool.pop()
                    else:
                        print("DRAW")
                        draw_tile = self.wall.tiles[0]
                        self.wall.tiles = self.wall.tiles[1:]
                        self.current_player.draw(draw_tile)
                    print('Pool:')
                    print(self.pool)
                    print('\n')
                        
                
                # throw event
                print('Joker:')
                print(self.joker)
                print('\n')
                thrown_tile = self.current_player.throw()
                self.pool.append(thrown_tile)
                print('{} throws out:'.format(self.current_player.name))
                print(''' 
                 _____
                |  {}  |
                |     |
                |  {}  |
                |_____|'''.format(thrown_tile[0], thrown_tile[1]))
                print('\n')
                    
                

## Test

In [5]:
mahjong = MahJong(player_names=['Tom', 'Helen', 'Hannah', 'Brenda'], tiles=create_tiles())

MAHJONG - created by Tom Sharp


SETTING UP MAHJONG TABLE
Seating Order: ['Hannah', 'Tom', 'Brenda', 'Helen']
Tiles: [('B', 1), ('B', 1), ('B', 1), ('B', 1), ('B', 2), ('B', 2), ('B', 2), ('B', 2), ('B', 3), ('B', 3), ('B', 3), ('B', 3), ('B', 4), ('B', 4), ('B', 4), ('B', 4), ('B', 5), ('B', 5), ('B', 5), ('B', 5), ('B', 6), ('B', 6), ('B', 6), ('B', 6), ('B', 7), ('B', 7), ('B', 7), ('B', 7), ('B', 8), ('B', 8), ('B', 8), ('B', 8), ('B', 9), ('B', 9), ('B', 9), ('B', 9), ('C', 1), ('C', 1), ('C', 1), ('C', 1), ('C', 2), ('C', 2), ('C', 2), ('C', 2), ('C', 3), ('C', 3), ('C', 3), ('C', 3), ('C', 4), ('C', 4), ('C', 4), ('C', 4), ('C', 5), ('C', 5), ('C', 5), ('C', 5), ('C', 6), ('C', 6), ('C', 6), ('C', 6), ('C', 7), ('C', 7), ('C', 7), ('C', 7), ('C', 8), ('C', 8), ('C', 8), ('C', 8), ('C', 9), ('C', 9), ('C', 9), ('C', 9), ('D', 1), ('D', 1), ('D', 1), ('D', 1), ('D', 2), ('D', 2), ('D', 2), ('D', 2), ('D', 3), ('D', 3), ('D', 3), ('D', 3), ('D', 4), ('D', 4), ('D', 4), ('D', 4), ('

In [6]:
mahjong.create_game()

CREATING GAME
Randomly shuffling tiles, building wall...




In [7]:
mahjong.game.deal_tiles()

DEALING TILES
Dealing tiles to each player, removing winds...




In [8]:
mahjong.game.reveal_joker()

JOKER
 
                 _____
                |  B  |
                |     |
                |  8  |
                |_____|


In [ ]:
mahjong.game.start()

STARTING GAME
Dealer (Hannah): Throw out first tile...
Joker:
('B', 8)


Hand:
[('B', 1), ('B', 2), ('B', 6)]
[('C', 1), ('C', 5), ('C', 5)]
[('D', 1), ('D', 1), ('D', 3), ('D', 3), ('D', 3), ('D', 7), ('D', 8), ('D', 8), ('D', 9), ('D', 9), ('D', 9)]


Revealed:
[]


Which tile to throw out?B6
Hannah throws out:
 
             _____
            |  B  |
            |     |
            |  6  |
            |_____|


PONG! Helen
Joker:
('B', 8)


Hand:
[('B', 1), ('B', 2), ('B', 4), ('B', 4), ('B', 7), ('B', 8), ('B', 9)]
[('C', 2), ('C', 5)]
[('D', 2), ('D', 3), ('D', 4), ('D', 4), ('D', 9)]


Revealed:
[('B', 6, 'B', 6, 'B', 6)]


Which tile to throw out?C5
Helen throws out:
 
                 _____
                |  C  |
                |     |
                |  5  |
                |_____|


PONG! Hannah
Joker:
('B', 8)


Hand:
[('B', 1), ('B', 2)]
[('C', 1)]
[('D', 1), ('D', 1), ('D', 3), ('D', 3), ('D', 3), ('D', 7), ('D', 8), ('D', 8), ('D', 9), ('D', 9), ('D', 9)]


Revealed:
[(

### assert False

In [ ]:
mahjong.game.pool[-1]

In [ ]:
nums = [tile[1] for tile in mahjong.game.current_player.hand if tile[0]==mahjong.game.pool[-1][0]]
if mahjong.game.pool[-1][1] not in nums:
    print(nums)

In [ ]:
nums

In [ ]:
for pair in [(mahjong.game.pool[-1][1] - 1 , mahjong.game.pool[-1][1] + 1),
              (mahjong.game.pool[-1][1] - 2 , mahjong.game.pool[-1][1] - 1),
              (mahjong.game.pool[-1][1] + 2 , mahjong.game.pool[-1][1] + 1)]:
    if pair[0] in nums and pair[1] in nums:
        can_eat = True

In [ ]:
can_eat

In [ ]:
for i, player in enumerate(mahjong.game.players):
    if player == mahjong.game.current_player:
        print(i)

In [ ]:
i = [i for i, player in enumerate(mahjong.game.players) if player == mahjong.game.current_player][0]
if i == 3:
    i = 0
else:
    i = i+1

In [ ]:
assert False

In [ ]:
# each person gets chance to pong
# if pong -> go to new player, throw out tile
# else next person 
    # if can eat -> eat and throw
    # else -> draw and throw

In [ ]:
for player in mahjong.game.players:
    # print players hand
    for s in ['B', 'C', 'D']:
        print([tile for tile in player.hand if tile[0].find(s) != -1])
#     print(player.hand)
    print('\n')

In [ ]:
thrown_tile = ('C', 1)

In [ ]:
# check pong
for player in mahjong.game.players:
    matches = [i for i, tile in enumerate(player.hand) if tile[0]==thrown_tile[0] and tile[1]==int(thrown_tile[1])]
    if len(matches)==2:
        print('PONG!', player.name)
        
        # add to player's revealed, remove from hand
        player.revealed.append(thrown_tile*3)
        for index in sorted(matches, reverse=True):
            del player.hand[index]
        
        
        # remove thrown tile from pool
#         mahjong.game.pool.pop()
        
        # throw tile, update current player
        mahjong.game.throw()
        
        # update current player

In [ ]:
for player in mahjong.game.players:
    # print players hand
    for s in ['B', 'C', 'D']:
        print([tile for tile in player.hand if tile[0].find(s) != -1])
#     print(player.hand)
    print('\n')